In [ ]:
using Pkg 
# Pkg.instantiate()
Pkg.activate(".")  # activate the environment for this notebook
# load the packages
using Flux, Dates, BSON, Measures, JLD2, Noise, JSON, Plots
using Printf, Statistics, ProgressMeter, ParameterSchedulers, LaTeXStrings
using ParameterSchedulers: Stateful , Optimisers
pyplot()

# Get the data

In [ ]:
function collect_data(data, branch; noise_param = 0.1)   
    # branch is either 1 or 2, branch 1 is h0, branch 2 is h_s, u_c 
    # notice how the branch number is the same as the output size
    timex = data["times"][branch]
    if branch == 1
        q_in = data["q_in"][branch]
        q_out = data["q_out"][branch]
        ext_forc = [ q_in q_out] 
    else 
        wind = data["tau"][branch]
        ext_forc = wind 
    end

    # sizes
    n_times=length(data["solution"][branch])
    n_steps=n_times-1 # number of time steps in the data

    # Create inputs X and outputs Y
    n_in = 3
    n_out = branch

    X = zeros(Float32,n_in,n_times-1) 
    Y = zeros(Float32,n_out,n_times-1)

    for t in 1:n_times-1
        X[1:branch,t] .=  data["solution"][branch][t].* (1 +  noise_param * randn(Float32) )
        X[1+branch:end,t] .=  ext_forc[t]
        Y[:,t] .= data["solution"][branch][t+1]
    end
    
    # Normalize
    Xμ= mean(X, dims=2); Xσ = std(X,  dims=2)
    X = (X .- Xμ) ./ Xσ 
    Y = (Y .- Xμ[1:branch,:]) ./ Xσ[1:branch,:]
    return X, Y, timex
end

# Model Architecture

In [ ]:
function build_model(n_hidden, n_layers, act, branch)
    # Build a dense layers model recursively, choosing the number of layers and number of features per layer
    layers = []
    push!(layers, Dense(3, n_hidden, act))
    for _ in 2:n_layers
        push!(layers, Dense(n_hidden, n_hidden, act))
    end
    push!(layers, Dense(n_hidden, branch))

    return Chain(layers...)  
end

# Training

In [ ]:
function train_data(model_r, X, Y; batch_size = 32, n_epochs = 1000, decay_rate = 0.99, step = 5)
    # Create minibathches
    train_loader = Flux.DataLoader((X, Y), batchsize=batch_size, shuffle=true)
    # Initialize optimizer and scheduler
    learning_rate = 0.001 / (decay_rate^(n_epochs/step))

    optimizer = Flux.setup(Adam(learning_rate), model_r)
    lr_scheduler = Stateful(Step(learning_rate, decay_rate, step))
    # Training loop
    for epoch in 1:n_epochs
        # Perform one epoch of training
        # Loop over splits in the training data
        for (x_batch, y_batch) in train_loader
            loss, grads = Flux.withgradient(model_r) do m
                # Evaluate model and loss inside gradient context:
                y_hat = m(x_batch) # apply the model to the input batch
                Flux.mse(y_hat, y_batch) # compute the loss
            end
            Flux.update!(optimizer, model_r, grads[1])
        end
        # Update learning rate
        nextlr = ParameterSchedulers.next!(lr_scheduler)
        Optimisers.adjust!(optimizer, nextlr)
    end
    
    y_hat = model_r(X)
    return model_r , Flux.mse(y_hat, Y)
end 

# Unroll

In [ ]:
# unroll the model to get the output for the initial condition
function unroll(model_r, x0, ext_forc,Y, branch)
    # Set up
    n_steps = size(ext_forc,2) - 1
    outputs = zeros(Float32, branch, n_steps+1)
    outputs[:,1] .= x0[1:branch] # store the initial condition
    # unroll the model for nstep
    x = x0
    for t in 1:n_steps
        y = model_r(x)
        outputs[:,t+1] .= y
        y = reshape(y,branch,)
        x = append!(y, ext_forc[:,t+1])
    end

    return outputs, Flux.mse(outputs, Y)
end

# Plot

In [ ]:
function plot_unroll(Y_unroll, Y, time, branch)
    # Plot the unroll on the correct branch
    if branch == 1
        p1=plot(time[1:end-1]/24, Y_unroll[1,:], xlabel="time [days]", xticks = 0:60:maximum(time)/24, label="ML unroll", legend=true, title="Mean water level in lake")# ,st=:scatter)
        plot!(p1, time[2:end]/24, Y[1,:], xlabel="time [days]", label="Ground truth", legend=true, title="Mean water level in lake")
        compareplot1 = plot(p1,layout=(1,1), size=(1200,300))
        display(compareplot1)

    elseif branch == 2
        p2=plot(time[1:end-1], Y_unroll[1,:], xlabel="time [hours]", label="ML unroll", xticks = 0:5:maximum(time), legend=true, title="Surface slope in lake [m]")
        plot!(p2, time[2:end], Y[1,:], xlabel="time [hours]", label="Ground truth", legend=true, title="Surface slope in lake [m]")
      
        p3=plot(time[1:end-1], Y_unroll[2,:], xlabel="time [hours]", label="ML unroll", xticks = 0:5:maximum(time), legend=true, title="Velocity in lake [m/s]")
        plot!(p3, time[2:end], Y[2,:], xlabel="time [hours]", label="Ground truth", legend=true, title="Velocity in lake [m/s]")
        
        compareplot1 = plot(p2,p3,layout=(2,1), size=(1200,600))
        display(compareplot1)
    end
    return compareplot1
end


# Example

In [ ]:

data=load("Dataset/model_0d_lake_sep.jld2")
for branch in [1,2]
    X, Y, time  = collect_data(data, branch, noise_param = 0.1)
    x0 = X[:,1]
    ext_forc = X[1+branch:end,:]
    model_r = build_model(15, 4, swish, branch)
    # 3-4 hidden layers and swish with n_hidden 15 is good enough usually
    model_r , mse = train_data(model_r, X, Y, n_epochs = 1000, step = 5)
    @show mse
    outputs , mse = unroll(model_r, x0, ext_forc, Y, branch)
    @show mse
    plot_unroll(outputs, Y, time, branch)
end

# Test stability

Testing the stability of the model by checking the stability of the unroll.

(for 200 trials it took 35 mins,for 400 trials it took 68 mins)

train_stat and unroll_stat contain the info of all the trials
best_stat_t and best_stat_u contain the info of the best performing trials

In [ ]:
# data=load("Dataset/model_0d_lake_sep.jld2")

# trials = 400  
# train_stat = zeros(Float32,trials,2) 
# unroll_stat = zeros(Float32,trials,2)
# dummy_model = build_model(15, 4, swish, 1)
# best_stat_t  = [ (dummy_model, [1e100, 0]), (dummy_model, [1e100, 0]) ]
# best_stat_u = [ (dummy_model, [1e100, 0]), (dummy_model, [1e100, 0]) ]

# for branch in [1 , 2]
#     println("Branch $branch")
#     X, Y, time  = collect_data(data, branch, noise_param = 0.1)
#     x0 = X[:,1]
#     ext_forc = X[1+branch:end,:]
#     for t in 1:trials
#         model_r = build_model(15, 4, swish, branch)
#         # 3-4 hidden layers and swish with n_hidden 15 works good enough usually
        
#         model_r, mse = train_data(model_r, X, Y, n_epochs = 1000, step = 5)
#         train_stat[t,branch] = mse
#         if best_stat_t[branch][2][1] > mse
#             best_stat_t[branch] = (model_r, [mse, t])
#         end

#         outputs, mse = unroll(model_r, x0, ext_forc, Y, branch)
#         unroll_stat[t,branch] = mse
#         if best_stat_u[branch][2][1] > mse
#             best_stat_u[branch] = (model_r, [mse, t])
#         end
#     end
    
#     model_r = best_stat_u[branch][1]
#     outputs , mse = unroll(model_r, x0, ext_forc, Y, branch)
#     # plot_unroll(outputs, Y, time, branch)
#     @show best_stat_u[branch][2]
# end

# Test generality

Testing the performance of the best model obtained via the trials above on the storm datasets

In [ ]:
# @load "Dense plots/matrixtrain.jld2" train_stat
# @load "Dense plots/matrixunroll.jld2" unroll_stat
# @load "Dense plots/bestmodel_t.jld2" best_stat_t
# @load "Dense plots/bestmodel_u.jld2" best_stat_u

# for storm in [1,2,3,4]
#     for branch in 1:2
#         model_r = best_stat_u[branch][1]
#         @show best_stat_t[branch][2][1]
#         if storm == 1
#             data2 = load("Dataset/model_0d_lake_sep_odd1.jld2")
#         elseif storm == 2
#             data2 = load("Dataset/model_0d_lake_sep_odd2.jld2")
#         elseif storm == 3
#             data2 = load("Dataset/model_0d_lake_sep_odd3.jld2")
#         elseif storm == 4
#             data2 = load("Dataset/model_0d_lake_sep.jld2")
#         end 
#         X, Y, time  = collect_data(data2, branch, noise_param = 0.1)
#         x0 = X[:,1]
#         ext_forc = X[1+branch:end,:]
#         outputs, mse = unroll(model_r, x0, ext_forc, Y, branch)
#         @show storm, branch, mse
#         p = plot_unroll(outputs, Y, time, branch)    
#         savefig(p, "unroll_storm$(storm)_branch$(branch) Dense.png")
#     end
# end